In [1]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from imblearn.metrics import specificity_score

#from sklearnex import patch_sklearn
#patch_sklearn()

In [2]:
pd.set_option('display.max_columns', 200)

In [3]:
def percent_of_nan_rows(dataset:pd.DataFrame, column:str) -> float:
    return len(dataset[dataset[column].isna()]) / len(dataset) * 100

In [4]:
def weighed_specificity_score(y_test, y_pred):
    return specificity_score(y_test, y_pred, average='weighted')

In [5]:
#Attempted to implement - unknown error raised
def weighed_roc_auc_score(y_test, y_pred):
    return roc_auc_score(y_test, y_pred, average='weighted', multi_class='ovr')

In [6]:
def PerformGridSearchCV(data:pd.DataFrame, *, feature_cols:list, label_col:str, label_col_type:str,
                        train_row_len:int = 1200, model_name:str, cv:int):
    __docstring__ = 'Model can be "xgb", "rf", "dt", "nb".\nLabec col can be categorical "cat" or binary "bin".'
    X_train, X_test, y_train, y_test = (data[feature_cols][:train_row_len],
                                        data[feature_cols][train_row_len:],
                                        data[label_col][:train_row_len],
                                        data[label_col][train_row_len:])
    y_train = y_train.to_numpy().ravel()
    y_test = y_test.to_numpy().ravel()
    if model_name == 'xgb':
        model = GradientBoostingClassifier()
        param_grid = [
    {'n_estimators':[10, 100, 200, 500],
     'learning_rate':[1e-3, 1e-2, 1e-1, 1],
     'max_depth':[1, 2, 3],
    }
    ]
    elif model_name == 'rf':
        model = RandomForestClassifier()
        param_grid = [
    {'n_estimators':[10, 100, 200, 500],
     'criterion':['gini', 'entropy', 'log_loss'],
     'bootstrap': [True, False],
    }
    ]
    elif model_name == 'dt':
        model = DecisionTreeClassifier()
        param_grid = [
    {'criterion':['gini', 'entropy'],
    'class_weight':[None, 'balanced'],
    }
    ]
    elif model_name == 'nb':
        model = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
        param_grid = [{
            'verbose':[False]
        }]
    else:
        raise ValueError(f'Model should be in ["xgb", "rf", "dt", "nb"], {model} given.')
    
    if label_col_type == 'cat':
        specificity_score_ = make_scorer(weighed_specificity_score, greater_is_better=True)
        roc_auc_score_ = make_scorer(weighed_roc_auc_score, greater_is_better=True)
        scoring = {'accuracy':get_scorer('accuracy'),
                   'recall':get_scorer('recall_weighted'),
                   'f1':get_scorer('f1_weighted'),
                   'specificity_score':specificity_score_,
                   #'roc_auc_score':roc_auc_score_,
                  }
    elif label_col_type == 'bin':
        specificity_score_ = make_scorer(specificity_score, greater_is_better=True)
        scoring = {'accuracy':get_scorer('accuracy'),
                   'recall':get_scorer('recall'), 'f1':get_scorer('f1'), 'specificity_score':specificity_score_}
    else:
        raise ValueError(f'Label type should be in ["cat", "bin"], {label_col_type} given.')
    cols = list(map(lambda x: 'mean_test_'+str(x), scoring))
    cols.extend(['mean_fit_time', 'params'])
    cv_obj = GridSearchCV(model, param_grid,
                              cv=cv,
                              scoring=scoring, verbose=0,
                              n_jobs = -1,
                              refit=False,
                              )
    cv_obj.fit(X_train, y_train)
    df = pd.DataFrame(cv_obj.cv_results_)[cols]
    df['model'] = model_name
    df['label_column'] = label_col
    return df

In [7]:
def get_model(model_name:str, params:dict):
    if model_name == 'xgb':
        model = GradientBoostingClassifier(**params)
    elif model_name == 'rf':
        model = RandomForestClassifier(**params)
    elif model_name == 'dt':
        model = DecisionTreeClassifier(**params)
    elif model_name == 'nb':
        model = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())], **params)
    return model

In [8]:
#Load dataset
myocardial_infarction_complications = fetch_ucirepo(id=579)
X = myocardial_infarction_complications.data.features
y = myocardial_infarction_complications.data.targets

In [9]:
#Assure that all NaN values are np.nan
X = X.fillna(np.nan)

In [10]:
#Get data on columns
vars_ = pd.DataFrame(myocardial_infarction_complications.variables)

In [11]:
#Number of missing values and number of complete rows
print(f'Missing values amount: {X.isna().sum().sum()}')
print(f'Fully entered rows amount: {len(X.dropna(axis=0, how="any"))}')

Missing values amount: 15974
Fully entered rows amount: 0


In [12]:
#Drop rows with more than 20% of values are NaN
X.drop(index = X[X.count(axis=1)<len(X.columns)*0.8].index, inplace=True)

In [13]:
#Drop columns with more than 30% of values are NaN
#Note: labels columns do not contain NaN values
for column in X.columns:
    if percent_of_nan_rows(X, column) > 30:
        X.drop(columns = [column], inplace=True)
        vars_.drop(index = vars_.loc[vars_['name'] == column].index, axis = 0, inplace=True)

In [14]:
#Get column names for columns of categorical, binary, continuous and int variable types
cat_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Categorical')]['name'].tolist()
bin_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Binary')]['name'].tolist()
int_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Integer')]['name'].tolist()
float_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Continuous')]['name'].tolist()

In [15]:
bin_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Binary')]['name'].tolist()
cat_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Categorical')]['name'].tolist()

In [16]:
#comment out
#y = pd.get_dummies(data = y, columns = cat_cols_preds)

In [17]:
data = X.merge(y, left_index=True, right_index=True)

In [18]:
#Multivariate iterative imputing of missing variables
imp = IterativeImputer(max_iter=100)

In [19]:
#Number of missing values and number of complete rows
print(f'Missing values amount: {data.isna().sum().sum()}')
print(f'Fully entered rows amount: {len(data.dropna(axis=0, how="any"))}')

Missing values amount: 4473
Fully entered rows amount: 544


In [20]:
imp.fit(data)

IterativeImputer(max_iter=100)

In [21]:
#Applying multivariate iterative imputing of missing variables /w rounding of values (needed for categorical data)
imp_data = pd.DataFrame(np.round(imp.transform(data)), columns=data.columns)

In [22]:
imp_data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,MP_TP_POST,SVT_POST,GT_POST,FIB_G_POST,ant_im,lat_im,inf_im,post_im,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,R_AB_1_n,R_AB_2_n,R_AB_3_n,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_1_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,77.0,1.0,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,138.0,0.0,0.0,8.0,16.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,132.0,0.0,0.0,8.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,132.0,0.0,0.0,11.0,10.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,68.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,120.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,146.0,1.0,0.0,9.0,14.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,60.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,132.0,0.0,0.0,8.0,16.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,70.0,0.0,0.0,2.0,2.0,1.0,2.0,0.0,

In [23]:
#Standardize int and float columns
for column in imp_data.columns:
    if (column in int_cols or column in float_cols):
        imp_data[column] = imp_data[column].map(
            lambda l: (l-imp_data[column].mean())/imp_data[column].std())

In [24]:
#Getting correlation matrix
corr_matrix = imp_data.corr(method='spearman')

In [25]:
identity = np.identity(n=len(corr_matrix.values))

In [26]:
where_ = np.where(corr_matrix.values-identity > 0.8)

In [27]:
where_ = np.vstack([where_[0], where_[1]])

In [28]:
#Displaying columns with significant correlation
for i in range(len(where_[0]))[::2]:
    print(corr_matrix.iloc[where_[0][i]].index[where_[0][i]], end='\t')
    print(corr_matrix.columns[where_[1][i]], end='\t')
    print(corr_matrix.iloc[where_[0][i]][where_[1][i]])

STENOK_AN	FK_STENOK	0.8450335438356769
S_AD_ORIT	D_AD_ORIT	0.8331132905991112
MP_TP_POST	ritm_ecg_p_02	0.845850627969873


/tmp/ipykernel_72713/2935923283.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(corr_matrix.iloc[where_[0][i]][where_[1][i]])


In [29]:
#Miocardial rupture leads to death in 100% cases - RAZRIV is not needed as a label (repeats LET_IS_3)
imp_data.drop(columns = 'RAZRIV', inplace=True)
vars_.drop(index = vars_.loc[vars_['name'] == 'RAZRIV'].index, axis = 0, inplace=True)

In [30]:
#Paroxysms of atrial fibrillation at the time of admission ==
#ECG rhythm at the time of admission to hospital: atrial fibrillation - can be dropped to reduce dimensionality
imp_data.drop(columns = 'MP_TP_POST', inplace=True)
vars_.drop(index = vars_.loc[vars_['name'] == 'MP_TP_POST'].index, axis = 0, inplace=True)

In [31]:
imp_data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,SVT_POST,GT_POST,FIB_G_POST,ant_im,lat_im,inf_im,post_im,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,R_AB_1_n,R_AB_2_n,R_AB_3_n,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_1_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1.356955,1.0,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.632210,1.059206,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.116897,0.0,0.245763,-0.672061,-0.302823,-0.234990,0.227566,4.0,0.0,0.0,1.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.581310,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.567952,0.417712,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,-0.234990,-0.969282,2.0,0.0,0.0,0.0,0.0,-0.621742,-0.261379,-0.203819,1.0,-0.279280,-0.236833,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.845619,1.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.632210,1.059206,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,0.665709,-0.324825,3.0,3.0,0.0,0.0,0.0,0.659523,-0.261379,-0.203819,3.0,4.757454,5.403772,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.564028,0.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.567952,-0.865277,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,1.605905,1.141806,-0.302823,0.065243,0.043436,2.0,0.0,0.0,1.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.140795,1.0,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.898823,0.417712,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,-0.234990,0.227566,9.0,0.0,0.0,0.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [32]:
#Arterial hypertension presence correlating to each other
#imp_data[['GB', 'DLIT_AG']]
#Exertional angina pectoris == Functional class (FC) of angina pectoris correlation of presence too
#imp_data[['STENOK_AN', 'FK_STENOK']]

In [33]:
imp_data = pd.get_dummies(data = imp_data, columns = cat_cols).sample(frac=1)

In [34]:
imp_data

,AGE,SEX,SIM_GIPERT,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,SVT_POST,GT_POST,FIB_G_POST,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS,INF_ANAM_0.0,INF_ANAM_1.0,INF_ANAM_2.0,INF_ANAM_3.0,STENOK_AN_0.0,STENOK_AN_1.0,STENOK_AN_2.0,STENOK_AN_3.0,STENOK_AN_4.0,STENOK_AN_5.0,STENOK_AN_6.0,FK_STENOK_0.0,FK_STENOK_1.0,FK_STENOK_2.0,FK_STENOK_3.0,FK_STENOK_4.0,IBS_POST_0.0,IBS_POST_1.0,IBS_POST_2.0,GB_0.0,GB_1.0,GB_2.0,GB_3.0,DLIT_AG_0.0,DLIT_AG_1.0,DLIT_AG_2.0,DLIT_AG_3.0,DLIT_AG_4.0,DLIT_AG_5.0,DLIT_AG_6.0,DLIT_AG_7.0,DLIT_AG_8.0,DLIT_AG_9.0,ZSN_A_0.0,ZSN_A_1.0,ZSN_A_2.0,ZSN_A_3.0,ZSN_A_4.0,ant_im_0.0,ant_im_1.0,ant_im_2.0,ant_im_3.0,ant_im_4.0,lat_im_0.0,lat_im_1.0,lat_im_2.0,lat_im_3.0,lat_im_4.0,inf_im_-1.0,inf_im_0.0,inf_im_1.0,inf_im_2.0,inf_im_3.0,inf_im_4.0,post_im_0.0,post_im_1.0,post_im_2.0,post_im_3.0,post_im_4.0,TIME_B_S_1.0,TIME_B_S_2.0,TIME_B_S_3.0,TIME_B_S_4.0,TIME_B_S_5.0,TIME_B_S_6.0,TIME_B_S_7.0,TIME_B_S_8.0,TIME_B_S_9.0,R_AB_1_n_0.0,R_AB_1_n_1.0,R_AB_1_n_2.0,R_AB_1_n_3.0,R_AB_2_n_0.0,R_AB_2_n_1.0,R_AB_2_n_2.0,R_AB_2_n_3.0,R_AB_3_n_0.0,R_AB_3_n_1.0,R_AB_3_n_2.0,R_AB_3_n_3.0,NOT_NA_1_n_0.0,NOT_NA_1_n_1.0,NOT_NA_1_n_2.0,NOT_NA_1_n_3.0,NOT_NA_1_n_4.0
1326,1.180749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.768114,-2.789761,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,1.605905,1.141806,3.196462,0.065243,2.989525,0.0,1.940787,-0.261379,-0.203819,-0.279280,-0.236833,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,True,False,False,False
1004,0.035411,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.567952,-0.223783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.485601,0.0,1.605905,-0.672061,-0.302823,-0.835456,-0.693086,0.0,-0.621742,-0.261379,-0.203819,-0.279280,-0.236833,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False
558,-2.431472,1.0,0.0,0.0,0.0,0.0,0.0,0.

In [35]:
#Features and labels for classification
features = imp_data.columns[~imp_data.columns.isin(y.columns)]

In [36]:
labels = imp_data.columns[imp_data.columns.isin(y.columns)]

In [37]:
#Attemptimg PCA on dummy dataset
pca = PCA(0.9, svd_solver='full')
imp_data_pcanalysed = pca.fit_transform(imp_data[features])
imp_data_pcanalysed

array([[ 4.00844272e+00,  1.30356060e+00, -1.83427966e+00, ...,
        -5.73760907e-02, -1.65086003e-01, -2.18195564e-01],
       [ 2.86457777e-01, -1.21103078e+00, -3.39503895e-01, ...,
         2.95833574e-02,  3.29148038e-03, -6.99173783e-02],
       [-6.75430534e-01, -1.11849920e+00,  6.18829164e-01, ...,
        -1.59916756e-04,  2.81551715e-02, -3.95039752e-02],
       ...,
       [ 8.07191624e-01,  1.62686056e-01, -1.87192279e+00, ...,
        -1.80120895e-01, -5.95166167e-01,  1.19231134e-01],
       [ 6.39884104e-01, -1.15171100e+00,  1.57394751e+00, ...,
        -1.25261838e-01,  5.18722009e-01,  6.54142221e-02],
       [ 2.73504084e-02,  6.41275863e-02, -9.38098290e-01, ...,
        -5.67054347e-01,  4.44551991e-01, -4.07983150e-01]])

In [38]:
imp_data_pcanalysed.shape
#51-dimensional data - 3 times reduction, but the curse of dimensionality is not lifted

(1560, 51)

In [39]:
imp_data_pcanalysed_train = imp_data_pcanalysed[:1200]
imp_data_pcanalysed_test = imp_data_pcanalysed[1200:]

In [40]:
#To avoid the need for multi-label classification, the model will be trained on one label at a time
#Starting with Letality prediction

In [41]:
labels = labels[labels.str.startswith('LET_IS')][0]

In [42]:
#Train-test split on shuffled data w/o sklearn
X_train, X_test, y_train, y_test = imp_data[features][:1200], imp_data[features][1200:], imp_data[labels][:1200], imp_data[labels][1200:]
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

In [43]:
#The best classification algorithm will be searched via Grid search cross-validation
#It allows the testing of best parameters for any classifier
#Classifiers tested will be:
'''
xGB
Random Forests
Decision Trees
Multinomial Naive Bayes
'''
model_list = ['xgb', 'rf', 'dt', 'nb']
#Adding other models is not recommended - addition of class instances required to all functions

In [44]:
cv_ = 6

In [45]:
out = pd.DataFrame()
for model in model_list:
    res = PerformGridSearchCV(imp_data, feature_cols=features, label_col=labels, label_col_type='cat',
                             model_name=model, cv=cv_)
    out = pd.concat([out, res], ignore_index=True, axis=0)

/home/eugen/anaconda3/envs/tfv2/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/eugen/anaconda3/envs/tfv2/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/home/eugen/anaconda3/envs/tfv2/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_72713/1927810886.py", line 2, in weighed_roc_auc_score
  File "/home/eugen/anaconda3/envs/tfv2/lib/python3.11/site-packages/sklearn/utils/_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/eugen/anaconda

In [46]:
first_metric = 'mean_test_specificity_score'
sec_metric = 'mean_test_f1'

In [47]:
#Employ a sum of 2 metrics - both F1 and specificity are equaly important
metric_sum = 'sum_f1_spec'
out[metric_sum] = out[first_metric]+out[sec_metric]
out = out.sort_values(metric_sum, ascending=False)

In [48]:
out

,mean_test_accuracy,mean_test_recall,mean_test_f1,mean_test_specificity_score,mean_test_roc_auc_score,mean_fit_time,params,model,label_column,sum_f1_spec
44,0.770833,0.770833,0.775669,0.530444,NaN,0.274903,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",xgb,LET_IS,1.306113
76,0.841667,0.841667,0.810600,0.475641,NaN,0.016101,{'verbose': False},nb,LET_IS,1.286241
40,0.606667,0.606667,0.627727,0.638857,NaN,0.195202,"{'learning_rate': 1, 'max_depth': 2, 'n_estima...",xgb,LET_IS,1.266584
46,0.625000,0.625000,0.648289,0.604620,NaN,4.885947,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",xgb,LET_IS,1.252909
27,0.836667,0.836667,0.807070,0.439355,NaN,5.802933,"{'learning_rate': 0.1, 'max_depth': 1, 'n_esti...",xgb,LET_IS,1.246425
...,...,...,...,...,...,...,...,...,...,...
20,0.841667,0.841667,0.769308,0.158333,NaN,0.281115,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",xgb,LET_IS,0.927641
5,0.841667,0.841667,0.769308,0.158333,NaN,1.821964,"{'learning_rate': 0.001, 'max_depth': 2, 'n_es...",xgb,LET_IS,0.927641
4,0.841667,0.841667,0.769308,0.158333,NaN,0.219183,"{'learning_rate': 0.001, 'max_depth': 2, 'n_es...",xgb,LET_IS,0.927641
2,0.841667,0.841667,0.769308,0.158333,NaN,2.336123,"{'learning_rate': 0.001, 'max_depth': 1, 'n_es...",xgb,LET_IS,0.927641


In [49]:
model_name, params = out.loc[0][['model' ,'params']]

In [50]:
model = get_model(model_name, params)

In [51]:
y_pred = model.fit(X_train, y_train).predict(X_test)

In [52]:
#F1 score for predicting mortality
f1_score(y_test, y_pred, average='weighted')

0.7928998505231688

In [53]:
#Low specificity due to class imbalance
specificity_score(y_test, y_pred, average='weighted')

0.14166666666666666

In [54]:
#Binary variables now
labels = imp_data.columns[imp_data.columns.isin(y.columns)]
labels = labels[~labels.str.startswith('LET_IS')]

In [55]:
#Reduce to 3 since dataset is too imbalanced - some values do not appear in entire fractions of the train set
cv_ = 3

In [56]:
out = pd.DataFrame()
for label in labels:
    for model in model_list:
        res = PerformGridSearchCV(imp_data, feature_cols=features, label_col=label, label_col_type='bin',
                                 model_name=model, cv=cv_)
        out = pd.concat([out, res], ignore_index=True, axis=0)
#Precision is ill-defined and being set to 0.0 due to no predicted samples. - some GridSearchCV instances
# only contain "False" label values, so the TP/P ratio cannot be calculated due to profound set imbalance

In [57]:
metric_sum = 'sum_f1_spec'
out[metric_sum] = out[first_metric]+out[sec_metric]
out = out.sort_values(metric_sum, ascending=False)

In [58]:
#Max F1 score is up to ~0.47 - too low
out

,mean_test_accuracy,mean_test_recall,mean_test_f1,mean_test_specificity_score,mean_fit_time,params,model,label_column,sum_f1_spec
572,0.797500,0.319149,0.425196,0.944444,0.241820,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",xgb,ZSN,1.369640
569,0.796667,0.304965,0.412761,0.947712,0.350538,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",xgb,ZSN,1.360473
562,0.801667,0.273050,0.392399,0.964052,1.227992,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",xgb,ZSN,1.356452
568,0.800000,0.280142,0.395487,0.959695,0.186135,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",xgb,ZSN,1.355182
586,0.775833,0.425532,0.471297,0.883442,0.883936,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",xgb,ZSN,1.354739
...,...,...,...,...,...,...,...,...,...
349,0.950000,0.000000,0.000000,0.981912,0.174546,"{'learning_rate': 1, 'max_depth': 2, 'n_estima...",xgb,A_V_BLOK,0.981912
227,0.961667,0.000000,0.000000,0.981293,0.005251,"{'class_weight': None, 'criterion': 'entropy'}",dt,JELUD_TAH,0.981293
199,0.960833,0.000000,0.000000,0.980442,0.214166,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",xgb,JELUD_TAH,0.980442
151,0.966667,0.000000,0.000000,0.977258,0.004460,"{'class_weight': 'balanced', 'criterion': 'gini'}",dt,PREDS_TAH,0.977258


In [59]:
#Testing PCA prediction F1 score for predicting mortality
model = get_model(model_name, params)
y_pred = model.fit(imp_data_pcanalysed_train, y_train).predict(imp_data_pcanalysed_test)
f1_score(y_test, y_pred, average='weighted')
#It is lower than that of the dataset without SVD dimensionality reduction - PCA is detremental to scoring
#(may however improve fit and pred times)

0.7928998505231688

In [60]:
specificity_score(y_test, y_pred, average='weighted')

0.14166666666666666

In [61]:
#Max values for separate variables - bad overall
out.groupby('label_column').max('mean_test_f1')

,mean_test_accuracy,mean_test_recall,mean_test_f1,mean_test_specificity_score,mean_fit_time,sum_f1_spec
label_column,,,,,,
A_V_BLOK,0.969167,0.282051,0.244667,1.0,1.250693,1.212798
DRESSLER,0.954167,0.091618,0.090236,1.0,1.226614,1.049916
FIBR_JELUD,0.956667,0.171024,0.207726,1.0,1.241391,1.185052
FIBR_PREDS,0.899167,0.229472,0.200579,1.0,1.167853,1.148402
JELUD_TAH,0.980000,0.208333,0.121327,1.0,1.178106,1.080511
OTEK_LANC,0.910000,0.279279,0.290482,1.0,1.212647,1.223448
PREDS_TAH,0.989167,0.083333,0.133333,1.0,1.171088,1.129122
P_IM_STEN,0.915833,0.178253,0.161632,1.0,1.186306,1.066996
REC_IM,0.912500,0.161905,0.175381,1.0,1.196361,1.147070
